# CRUD separado por roles


## Estructura de CRUD

In [ ]:
.
├── create.php
├── read.php
├── update.php
├── delete.php
├── index.php
├── db.php
├── css
│   └── styles.css
└── js
    └── script.js


### Código conexión a base de datos

In [ ]:
<?php
// Archivo: db.php
// Descripción: Este archivo se encarga de establecer la conexión con la base de datos MySQL.

// Configuración de la conexión
$host = 'localhost';            // Dirección del servidor de base de datos.
$user = 'root';                 // Usuario de la base de datos.
$password = '';                 // Contraseña del usuario.
$database = 'hotel_reservas';   // Nombre de la base de datos.

// Crear la conexión con la base de datos
$conn = mysqli_connect($host, $user, $password, $database);

// Verificar si la conexión fue exitosa
if (!$conn) {
    die("Error de conexión: " . mysqli_connect_error()); // Corregido mysqli_conn a mysqli_connect_error
}

// Nota: No cerramos la conexión aquí, se cierra después de usarla en cada script.
?>



### Código conexión a CREATE del CRUD

In [ ]:
//create.php
<?php
// Archivo: create.php
require 'db.php'; // Incluir conexión a la base de datos

if ($_SERVER['REQUEST_METHOD'] == 'POST') {
    // Sanitizar los datos del formulario
    $nombre = mysqli_real_escape_string($conn, $_POST['nombre']);
    $apellido = mysqli_real_escape_string($conn, $_POST['apellido']);
    $telefono = mysqli_real_escape_string($conn, $_POST['telefono']);
    $habitacion = mysqli_real_escape_string($conn, $_POST['habitacion']);
    $fecha_entrada = $_POST['fecha_entrada'];
    $fecha_salida = $_POST['fecha_salida'];
    $precio = floatval($_POST['precio']);

    // Consulta SQL para insertar los datos
    $sql = "INSERT INTO reservaciones (nombre, apellido, telefono, habitacion, fecha_entrada, fecha_salida, precio) 
            VALUES ('$nombre', '$apellido', '$telefono', '$habitacion', '$fecha_entrada', '$fecha_salida', '$precio')";

    // Ejecutar la consulta
    if (mysqli_query($conn, $sql)) {
        echo json_encode(['status' => 'success', 'message' => 'Reservación registrada exitosamente']);
        header('Location: index.php?mensaje=Reservación registrada exitosamente');
    } else {
        echo json_encode(['status' => 'error', 'message' => 'Error al registrar: ' . mysqli_error($conn)]);
    }
}

// Cerrar la conexión
mysqli_close($conn);
?>


### Código conexión a READ del CRUD

In [ ]:
//read.php
<?php
require 'db.php';

$consulta_hotel = mysqli_query($conn, "SELECT * FROM reservaciones");

if (mysqli_num_rows($consulta_hotel) > 0) {
    while ($row = mysqli_fetch_assoc($consulta_hotel)) {
        echo "<tr>
                <td>{$row['id']}</td>
                <td>{$row['nombre']}</td>
                <td>{$row['apellido']}</td>
                <td>{$row['telefono']}</td>
                <td>{$row['habitacion']}</td>
                <td>{$row['fecha_entrada']}</td>
                <td>{$row['fecha_salida']}</td>
                <td>{$row['precio']}</td>
                <td>
                    <button onclick='eliminarReservacion({$row['id']})'>Eliminar</button>
                    <button onclick='mostrarFormularioActualizar({$row['id']}, \"{$row['nombre']}\", \"{$row['apellido']}\", \"{$row['telefono']}\", \"{$row['habitacion']}\", \"{$row['fecha_entrada']}\", \"{$row['fecha_salida']}\", \"{$row['precio']}\")'>Actualizar</button>
                </td>
            </tr>";
    }
} else {
    echo "<tr><td colspan='9'>No hay reservaciones registradas.</td></tr>";
}
?>


### Código conexión a UPDATE del CRUD

In [ ]:
//update.php
<?php
require 'db.php';

if (isset($_POST['update'])) {
    $update_id = $_POST['update_id'];
    $update_nombre = $_POST['update_nombre'];
    $update_apellido = $_POST['update_apellido'];
    $update_telefono = $_POST['update_telefono'];
    $update_habitacion = $_POST['update_habitacion'];
    $update_fecha_entrada = $_POST['update_fecha_entrada'];
    $update_fecha_salida = $_POST['update_fecha_salida'];
    $update_precio = $_POST['update_precio'];

    $update_query = "UPDATE reservaciones SET 
                    nombre = '$update_nombre', 
                    apellido = '$update_apellido', 
                    telefono = '$update_telefono', 
                    habitacion = '$update_habitacion', 
                    fecha_entrada = '$update_fecha_entrada', 
                    fecha_salida = '$update_fecha_salida', 
                    precio = '$update_precio' 
                    WHERE id = $update_id";

    if (mysqli_query($conn, $update_query)) {
        header("Location: index.php?mensaje=¡Reservación actualizada exitosamente!");
    } else {
        echo "Error al actualizar la reservación: " . mysqli_error($conn);
    }
}
?>

### Código conexión a DELETE del CRUD

In [ ]:
//delete.php
<?php
require 'db.php';

if (isset($_GET['delete_id'])) {
    $delete_id = $_GET['delete_id'];
    $delete_query = "DELETE FROM reservaciones WHERE id = $delete_id";

    if (mysqli_query($conn, $delete_query)) {
        header("Location: index.php?mensaje=¡Reservación eliminada exitosamente!");
    } else {
        echo "Error al eliminar la reservación: " . mysqli_error($conn);
    }
}
?>



### Código index.php

```html
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>CRUD de Reservaciones</title>
    <link rel="stylesheet" href="css/styles.css">
</head>
<body>
<div class="container">
    <h2>Crear Nueva Reservación</h2>
    <form action="create.php" method="POST" onsubmit="return validarFormulario()">
        <input type="text" name="nombre" placeholder="Nombre" required>
        <input type="text" name="apellido" placeholder="Apellido" required>
        <input type="text" name="telefono" placeholder="Teléfono" required>
        <input type="text" name="habitacion" placeholder="Habitación" required>
        <input type="date" name="fecha_entrada" required>
        <input type="date" name="fecha_salida" required>
        <input type="number" name="precio" placeholder="Precio en COP" required>
        <button type="submit">Cargar</button>
    </form>

    <h2>Reservaciones Existentes</h2>
    <table>
        <tr>
            <th>ID</th>
            <th>Nombre</th>
            <th>Apellido</th>
            <th>Teléfono</th>
            <th>Habitación</th>
            <th>Fecha de Entrada</th>
            <th>Fecha de Salida</th>
            <th>Precio</th>
            <th>Acciones</th>
        </tr>
        <?php include 'read.php'; ?>
    </table>
</div>
<script src="js/script.js"></script>
</body>
</html>


### Código CSS


```css
/* Estilos Generales */
body {
    font-family: Arial, sans-serif;
    background-color: #f4f4f4;
    margin: 0;
    padding: 0;
}

/* Cabecera */
header {
    background-color: #4CAF50;
    color: white;
    text-align: center;
    padding: 20px;
    font-size: 24px;
}

/* Tabla de Reservaciones */
table {
    width: 100%;
    border-collapse: collapse;
    margin: 20px 0;
}

table th, table td {
    padding: 12px;
    text-align: left;
    border: 1px solid #ddd;
}

table th {
    background-color: #4CAF50;
    color: white;
}

table tr:nth-child(even) {
    background-color: #f2f2f2;
}

button {
    background-color: #4CAF50;
    color: white;
    border: none;
    padding: 10px 20px;
    cursor: pointer;
    border-radius: 5px;
    font-size: 16px;
}

button:hover {
    background-color: #45a049;
}

button:disabled {
    background-color: #ddd;
    cursor: not-allowed;
}

/* Formulario */
form {
    background-color: white;
    padding: 20px;
    margin: 20px 0;
    border-radius: 8px;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    max-width: 500px;
    margin-left: auto;
    margin-right: auto;
}

form input[type="text"],
form input[type="date"],
form input[type="number"] {
    width: 100%;
    padding: 10px;
    margin: 10px 0;
    border: 1px solid #ddd;
    border-radius: 5px;
}

form button {
    width: 100%;
    padding: 12px;
    font-size: 18px;
    border-radius: 5px;
    cursor: pointer;
    background-color: #4CAF50;
}

form button[type="button"] {
    background-color: #f44336;
}

form button[type="button"]:hover {
    background-color: #e53935;
}

form button:hover {
    background-color: #45a049;
}

/* Estilo para la tabla de detalles y botones adicionales */
.btn-group {
    display: flex;
    justify-content: center;
    gap: 10px;
}

.btn-group button {
    background-color: #008CBA;
    color: white;
}

.btn-group button:hover {
    background-color: #007B8C;
}

/* Estilo para el mensaje de error/success */
.alert {
    padding: 10px;
    margin: 15px 0;
    border-radius: 5px;
    text-align: center;
    font-weight: bold;
}

.alert.success {
    background-color: #4CAF50;
    color: white;
}

.alert.error {
    background-color: #f44336;
    color: white;
}

/* Estilo para la actualización de reserva */
form.update-form {
    background-color: #ffffff;
    padding: 30px;
    margin: 20px 0;
    border-radius: 8px;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
}

.update-form input,
.update-form select {
    width: 100%;
    padding: 10px;
    margin: 10px 0;
    border: 1px solid #ddd;
    border-radius: 5px;
}

.update-form button {
    width: 100%;
    padding: 10px;
    background-color: #4CAF50;
    color: white;
    border: none;
    font-size: 16px;
    cursor: pointer;
}

.update-form button:hover {
    background-color: #45a049;
}

.update-form button[type="button"] {
    background-color: #f44336;
}

.update-form button[type="button"]:hover {
    background-color: #e53935;
}

/* Modificaciones para pantallas pequeñas */
@media (max-width: 768px) {
    table {
        font-size: 14px;
    }

    form {
        width: 90%;
    }

    .btn-group {
        flex-direction: column;
    }
}



### Codigo script.js

```js
//JAVASCRIPT

// Validar el formulario antes de enviar
function validarFormulario() {
    const nombre = document.querySelector('input[name="nombre"]').value.trim();
    const apellido = document.querySelector('input[name="apellido"]').value.trim();
    const telefono = document.querySelector('input[name="telefono"]').value.trim();
    const habitacion = document.querySelector('input[name="habitacion"]').value.trim();
    const fechaEntrada = document.querySelector('input[name="fecha_entrada"]').value;
    const fechaSalida = document.querySelector('input[name="fecha_salida"]').value;
    const precio = document.querySelector('input[name="precio"]').value.trim();

    if (!nombre || !apellido || !telefono || !habitacion || !fechaEntrada || !fechaSalida || !precio) {
        alert("Por favor, completa todos los campos.");
        return false;
    }

    if (isNaN(precio) || precio <= 0) {
        alert("El precio debe ser un número positivo.");
        return false;
    }

    if (new Date(fechaEntrada) >= new Date(fechaSalida)) {
        alert("La fecha de entrada debe ser anterior a la fecha de salida.");
        return false;
    }

    return true;
}

// Confirmación antes de eliminar una reservación
function eliminarReservacion(id) {
    if (confirm("¿Estás seguro de que deseas eliminar esta reservación?")) {
        window.location.href = `delete.php?delete_id=${id}`;
    }
}

// Mostrar el formulario de actualización con los datos cargados
function mostrarFormularioActualizar(id, nombre, apellido, telefono, habitacion, fecha_entrada, fecha_salida, precio) {
    const formActualizar = document.createElement('div');
    formActualizar.innerHTML = `
        <form action="update.php" method="POST">
            <input type="hidden" name="update_id" value="${id}">
            <input type="text" name="update_nombre" value="${nombre}" required>
            <input type="text" name="update_apellido" value="${apellido}" required>
            <input type="text" name="update_telefono" value="${telefono}" required>
            <input type="text" name="update_habitacion" value="${habitacion}" required>
            <input type="date" name="update_fecha_entrada" value="${fecha_entrada}" required>
            <input type="date" name="update_fecha_salida" value="${fecha_salida}" required>
            <input type="number" name="update_precio" value="${precio}" required>
            <button type="submit" name="update">Actualizar</button>
            <button type="button" onclick="cerrarFormularioActualizar()">Cancelar</button>
        </form>
    `;

    document.body.appendChild(formActualizar);
}

// Cerrar el formulario de actualización
function cerrarFormularioActualizar() {
    const formActualizar = document.querySelector('form[action="update.php"]');
    if (formActualizar) {
        formActualizar.remove();
    }
}


### Código SQL 


```sql
-- Crear base de datos
CREATE DATABASE IF NOT EXISTS hotel_reservas;

-- Usar la base de datos recién creada
USE hotel_reservas;

-- Crear la tabla reservaciones
CREATE TABLE IF NOT EXISTS reservaciones (
    id INT AUTO_INCREMENT PRIMARY KEY,    -- Identificador único de la reservación
    nombre VARCHAR(50) NOT NULL,          -- Nombre de la persona
    apellido VARCHAR(50) NOT NULL,        -- Apellido de la persona
    telefono VARCHAR(20) NOT NULL,        -- Teléfono de contacto
    habitacion VARCHAR(10) NOT NULL,      -- Número de habitación
    fecha_entrada DATE NOT NULL,          -- Fecha de entrada al hotel
    fecha_salida DATE NOT NULL,           -- Fecha de salida del hotel
    precio DECIMAL(10, 2) NOT NULL        -- Precio de la reservación en COP
);